# Fahrräder mit oder ohne Licht

In [ ]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np

from astral import LocationInfo
from astral.sun import sun

## Datenaufbereitung

In [ ]:
df = pd.read_csv('Fahrrad_Licht_ohneLicht_2021-04-03_a', delim_whitespace=True)
df

In [ ]:
#Behalten der Ursprungsdaten
df2 = df.copy()

In [ ]:
#Umbenennung der Spaltennamen in sinnvolle Bezeichnungen
df2.rename(columns={'Fahrrad': 'Datum', 'Licht:ohne': 'Uhrzeit', 'Licht': 'mit_ohne'}, inplace=True)

In [ ]:
#Entfernung des Häckchens und Ersetzung durch das Datum
df2.loc[df2.Datum=='[✓]','Datum']=df2['Uhrzeit']

In [ ]:
#Ersetzung des Datums in der Uhrzeit-Spalte durch die geplante Ankunfszeit
for index, value in enumerate(df2.Uhrzeit):
    if '2020' in value:
        df2['Uhrzeit'].iloc[index] = '08:30:00'

In [ ]:
#Trennung des Datums mit Uhrzeit aus 2020 in Datum und Uhrzeit
for index, value in enumerate(df2.Datum):
    if '-' in value:
        df2['Uhrzeit'].iloc[index] = value[-8:]
        df2['Datum'].iloc[index] = value[:10]

In [ ]:
#Auftrennung der Spalte mit Licht und ohne Licht 
df2 = df2.join(df2['mit_ohne'].str.split(':', expand=True).rename(columns={0:'mit_Licht', 1:'ohne_Licht'}))

In [ ]:
df2.info()

In [ ]:
df2.describe()

In [ ]:
df2['mit_Licht'] = pd.to_numeric(df2['mit_Licht'])
df2['ohne_Licht'] = pd.to_numeric(df2['ohne_Licht'])
df2['Datum'] = pd.to_datetime(df2['Datum'], format='%d/%m/%Y',errors='coerce').fillna(pd.to_datetime(df2['Datum'], format='%d.%m.%Y',errors='coerce'))
#pd.to_datetime(s, format="%d%m%Y", errors="coerce").fillna(pd.to_datetime(s, format="%d%b%Y", errors="coerce"))
df2['Uhrzeit'] = pd.to_datetime(df2['Uhrzeit'], format='%H:%M:%S').dt.time

In [ ]:
df2

In [ ]:
df2.info()

# Feature Engineering

In [ ]:
df2['Verhältnis'] = df2['ohne_Licht']/df2['mit_Licht']

In [ ]:
df2['Summe'] = df2['ohne_Licht'] + df2['mit_Licht']

In [ ]:
df2

### Sonnenaufgang

In [ ]:
#Berechnung des Sonnenaufgangs
df2['Sonnenaufgang'] = ''
city = LocationInfo("Karlsruhe", "Deutschland", "Europe/Berlin", 49.006889, 8.403653)
for index, value in enumerate(df2['Datum']):
    #print(index)
    s = sun(city.observer, date=value, tzinfo=city.timezone)
    df2['Sonnenaufgang'][index+1] = s['sunrise']

## Diagramme

In [ ]:
plt.plot('Datum', 'Verhältnis', data = df2)
plt.show()

In [ ]:
plt.plot('Datum', 'Summe', data = df2)
plt.show()